In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
df = pd.read_csv('ab_test_results_mar_2.csv')
df.shape

/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_36893/1153197093.py:1: DtypeWarning: Columns (12,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ab_test_results_mar_2.csv')


(16876, 41)

In [3]:
xdf = df[df['account_id'] == 'seattle']
model_df = pd.get_dummies(xdf[['account_id', 'sub_category', 'user', 'job_education']])
#model_df = pd.get_dummies(xdf[['user']])
model_df['is_new_algo'] = xdf['is_new_algo'].astype(float)
model_df['min_years_of_relevant_experience_scaled'] = xdf['min_years_of_relevant_experience_scaled'].fillna(1.0).astype(float)
#model_df =  lr_df[['is_new_algo']].astype(int)
model_df = sm.add_constant(model_df)

corr_matrix = model_df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.5)]
print('Removed:', to_drop)
model_df = model_df.drop(to_drop, axis=1)
model_df.shape

Removed: ['sub_category_Public_Health_Administration', 'user_376f39dd52a6010101cedd2b14840004', 'user_905bc4dee6b8010101d9fe3e15a20002', 'user_905bc4dee6b80101946cf3c578fe0000', 'job_education_high_school', 'job_education_none_required']


(386, 35)

In [4]:
logit = sm.Logit(xdf['is_positive'].astype(float), model_df.astype(float))

In [5]:

res = logit.fit(maxiter=200)
print(res.params['is_new_algo'], np.exp(res.params['is_new_algo']), res.pvalues['is_new_algo'])
print()

/Users/dima/.virtualenvs/hs/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/dima/.virtualenvs/hs/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


         Current function value: inf
         Iterations: 200


LinAlgError: Singular matrix

In [ ]:
model_df.shape

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
df['account_id'].value_counts()

In [ ]:
pd.get_dummies(xdf['user']).values.shape

In [ ]:
xdf = df[df['account_id'] == 'seattle']
#model_df = pd.get_dummies(xdf[['account_id', 'sub_category', 'user', 'job_education']])
users = PCA(n_components=3).fit_transform(pd.get_dummies(xdf['user']))
subs = PCA(n_components=3).fit_transform(pd.get_dummies(xdf['sub_category']))
accounts = PCA(n_components=1).fit_transform(pd.get_dummies(xdf['account_id']))
edus = PCA(n_components=3).fit_transform(pd.get_dummies(xdf['job_education']))


is_new_algo = xdf['is_new_algo'].astype(float).values.reshape(-1, 1)
min_years_of_relevant_experience_scaled = xdf['min_years_of_relevant_experience_scaled'].fillna(1.0).astype(float).values.reshape(-1, 1)
users.shape, subs.shape, accounts.shape, edus.shape, is_new_algo.shape, min_years_of_relevant_experience_scaled.shape

In [ ]:
X = np.hstack([users, subs, accounts, edus, min_years_of_relevant_experience_scaled, is_new_algo])
X.shape

In [ ]:
logit = sm.Logit(xdf['is_positive'].astype(float), X).fit()

In [ ]:
np.exp(logit.params['x14'])

In [ ]:
conf = ['account_id', 'top_category', 'user', 'job_education']
xdf = df[df['account_id'] != 'all']
model_df = pd.get_dummies(xdf[conf]).astype(int)
to_drop = []

for f in conf:
    to_drop.append([c for c in model_df.columns if f in c][0])

print('Dropping:', to_drop)
model_df = model_df.drop(to_drop, axis=1)
#model_df = pd.get_dummies(account_df[['account_id']])
model_df['is_new_algo'] = xdf['is_new_algo'].astype(int)
model_df['min_years_of_relevant_experience_scaled'] = xdf['min_years_of_relevant_experience_scaled'].fillna(1)
#model_df =  lr_df[['is_new_algo']].astype(int)
model_df = sm.add_constant(model_df)



res = sm.Logit(xdf['is_positive'], model_df, maxiter=200).fit()
print(f"OR={np.exp(res.params['is_new_algo'])}, p={res.pvalues['is_new_algo']}")
print()